# VariantSpark-k BigChr22

Parameter info - 50 executors, and 2GB per executor.  If using 100 trees, then ~3 min run, if using 1000 tree, then 10 min run.  Started with 500 trees - need to look at OOB for each.

Using 4 qty of r4.rxlarge EC2 instance & Kubernetes

### Parameters
 - input bucket location - Lynn's S3 bucket (us-west-2 / Oregon)
 - input files `ALL.chr22...` and fc `22_16051249`
 - number of Spark executors 50 w/ 2 GM RAM each
 - number of RF trees 100, w/ batch of 25
 - default `mtry`
 - calculate OOB as `-ro` paramater is configured

In [2]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark-llc

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."

spark-submit \
    --class au.csiro.variantspark.cli.VariantSparkApp \
    --driver-class-path ./conf \
    --master k8s://${MASTER} \
    --deploy-mode cluster \
    --name VariantSparkBig22 \
    --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
    --conf spark.executor.instances=50 \
    --conf spark.executor.memory=2g \
    --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
    --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
    local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
        -if s3a://${INPUT_BUCKET}/ALL.chr22.phase1_release_v3.20101123.snps_indels_svs.genotypes.vcf.bz2 \
        -ff s3a://${INPUT_BUCKET}/chr22-labels_release_v3.20101123.csv \
        -fc 22_16051249 \
        -v \
        -rn 100 \
        -rbs 25 \
        -ro "$@"


2018-08-05 03:13:03 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkbig22-f8bd969a19673a02a1717556f7ac994b-driver
	 namespace: default
	 labels: spark-app-selector -> spark-b4204b0727574913b31db4f1cff89619, spark-role -> driver
	 pod uid: 76df5298-985d-11e8-b501-0af7ed887422
	 creation time: 2018-08-05T03:13:02Z
	 service account name: spark
	 volumes: spark-init-properties, download-jars-volume, download-files-volume, spark-token-cv25b
	 node name: N/A
	 start time: N/A
	 container images: N/A
	 phase: Pending
	 status: []
2018-08-05 03:13:03 INFO  LoggingPodStatusWatcherImpl:54 - State changed, new state: 
	 pod name: variantsparkbig22-f8bd969a19673a02a1717556f7ac994b-driver
	 namespace: default
	 labels: spark-app-selector -> spark-b4204b0727574913b31db4f1cff89619, spark-role -> driver
	 pod uid: 76df5298-985d-11e8-b501-0af7ed887422
	 creation time: 2018-08-05T03:13:02Z
	 service account name: spark
	 volumes: spark-init-properties, download